目的实现连续和类别变量自动识别，单调分箱和无法单调分箱变量识别，根据判断情况进行woe分箱

In [ ]:
def woe_single(DF,Y,X):
    if X.nunique()>11:
        r = 0
        bad=Y.sum()      #坏客户数(假设因变量列为1的是坏客户)
        good=Y.count()-bad  #好客户数
        n=5
        while np.abs(r) < 1:
            d1 = pd.DataFrame({"X": X, "Y": Y, "Bucket": pd.qcut(X, n,duplicates='drop')})
            d2 = d1.groupby('Bucket', as_index = False)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1
        d3 = pd.DataFrame(d2.X.min(), columns = ['min'])
        d3['min']=d2.min().X    
        d3['max'] = d2.max().X
        d3['sum'] = d2.sum().Y
        d3['total'] = d2.count().Y
        d3['bad_rate'] = d2.mean().Y
        d3['group_rate']=d3['total']/(bad+good)
        d3['woe']=np.log((d3['bad_rate']/(1-d3['bad_rate']))/(bad/good))
        d3['iv']=(d3['sum']/bad-((d3['total']-d3['sum'])/good))*d3['woe']
        iv=d3['iv'].sum()
        if iv!=0.0 and len(d2)>1:
            d3['iv_sum']=iv
            woe=list(d3['woe'].round(6))
            cut=list(d3['min'].round(6))
            cut1=list(d3['max'].round(6))
            cut.append(cut1[-1]+1)
            x_woe=pd.cut(X,cut,right=False,labels=woe)
            return  d3,cut,woe,iv,x_woe
        else:
            dn1 = pd.DataFrame({"X": X, "Y": Y, "Bucket": pd.cut(X, 100)})
            dn2 = dn1.groupby('Bucket', as_index = False)
            dn3 = pd.DataFrame(dn2.X.min(), columns = ['min'])
            dn3['min']=dn2.min().X    
            dn3['max'] = dn2.max().X
            dn3['sum'] = dn2.sum().Y
            dn3['total'] = dn2.count().Y
            while (1):
                    if  (len(dn3)>4):
                        dn3_min_index = dn3[dn3.total == min(dn3.total)].index.values[0]
                        if (dn3_min_index!=0):    #最小值非第一行的情况
                            dn3.iloc[dn3_min_index-1, 1] =dn3.iloc[dn3_min_index, 1] 
                            dn3.iloc[dn3_min_index-1, 2] = dn3.iloc[dn3_min_index, 2] +dn3.iloc[dn3_min_index-1, 2]
                            dn3.iloc[dn3_min_index-1, 3] = dn3.iloc[dn3_min_index, 3] +dn3.iloc[dn3_min_index-1, 3]
                            dn3=dn3.drop([dn3_min_index])
                            dn3= dn3.reset_index(drop=True)
                        else:    #最小值是第一行的情况
                            dn3.iloc[dn3_min_index+1, 0] =dn3.iloc[dn3_min_index, 0] 
                            dn3.iloc[dn3_min_index+1, 2] = dn3.iloc[dn3_min_index, 2] +dn3.iloc[dn3_min_index+1, 2]
                            dn3.iloc[dn3_min_index+1, 3] = dn3.iloc[dn3_min_index, 3] +dn3.iloc[dn3_min_index+1, 3]
                            dn3=dn3.drop([dn3_min_index])
                            dn3= dn3.reset_index(drop=True)
                    else:
                        break
            dn3['bad_rate'] =dn3['sum']/dn3['total']
            dn3['group_rate']=dn3['total']/(bad+good)
            dn3['woe']=np.log((dn3['bad_rate']/(1-dn3['bad_rate']))/(bad/good))
            dn3['iv']=(dn3['sum']/bad-((dn3['total']-dn3['sum'])/good))*dn3['woe']
            
            iv=dn3['iv'].sum()
            dn3['iv_sum']=iv
            woe=list(dn3['woe'].round(6)) 
            cut=list(dn3['min'].round(6))
            cut1=list(dn3['max'].round(6))
            cut.append(cut1[-1]+1)
            x_woe=pd.cut(X,cut,right=False,labels=woe)
            return  dn3,cut,woe,iv,x_woe
    else : 
        bad=Y.sum()      #坏客户数
        good=Y.count()-bad  #好客户数
        d1 = pd.DataFrame({"X": X, "Y": Y})
        d2 = d1.groupby('X', as_index =True)
        d3 = pd.DataFrame()
        
        d3['sum'] = d2.sum().Y
        d3['total'] = d2.count().Y
        for c in range(d3.shape[0])[::-1]:
            if ((d3.iloc[c,1]-d3.iloc[c,0])==0) or (d3.iloc[c,0]==0):
                d3.iloc[c-1,0]=d3.iloc[c-1,0]+d3.iloc[c,0]
                d3.iloc[c-1,1]=d3.iloc[c-1,1]+d3.iloc[c,1]
                d3.drop(d3.index[c],inplace=True)
            else:
                continue
        
        d3['min']=d3.index  
        d3['max'] = d3.index
        d3['bad_rate'] =d3['sum']/d3['total']
        d3['group_rate']=d3['total']/(bad+good)
        d3['woe']=np.log((d3['bad_rate']/(1-d3['bad_rate']))/(bad/good))
        d3['iv']=(d3['sum']/bad-((d3['total']-d3['sum'])/good))*d3['woe']
        iv=d3['iv'].sum()
        d3['iv_sum']=iv
        d3=d3[['min','max','sum','total','bad_rate','group_rate','woe','iv','iv_sum']]
        
        
        woe=list(d3['woe'].round(6))
        cut=list(d3.index)
        x_woe=X.replace(cut,woe)

选取apply_request_sum这个变量来看下情况

In [ ]:
woe_single(df,df.overdue_7,df.apply_request_sum)[0]   #整体分箱WOE，IV情况

分割点

In [ ]:
woe_single(df,df.overdue_7,df.overdue_repay_sum)[1]

In [ ]:
woe_single(df,df.overdue_7,df.overdue_repay_sum)[2]   

iv值

In [ ]:
iv:woe_single(df,df.overdue_7,df.overdue_repay_sum)[3]

In [ ]:
woe=pd.DataFrame(columns=['min','max','sum','total','bad_rate','group_rate','woe','iv','iv_sum'])
for i in df.columns[5:]:
    w=pd.DataFrame(index=[i])
    w=pd.concat([w,woe_single(df,df.overdue_7,df[i])[0]])
    frames = [woe,w]
    woe=pd.concat(frames)
woe=woe[['min','max','sum','total','bad_rate','group_rate','woe','iv','iv_sum']]     #所有变量输出

#查看所有变量分箱、woe、iv等情况
woe

In [ ]:
#也可以导入Excel进行分析
#woe.to_excel('文件路径.xlsx')

转换原始数据为对应woe值

In [ ]:
def woe_all(DF,Y,X):
    for i in X:
        if DF[i].nunique()>11:
            r = 0
            bad=Y.sum()      #坏客户数(假设因变量列为1的是坏客户)
            good=Y.count()-bad  #好客户数
            n=5
            while np.abs(r) < 1:
                d1 = pd.DataFrame({"X": DF[i], "Y": Y, "Bucket": pd.qcut(DF[i], n,duplicates='drop')})
                d2 = d1.groupby('Bucket', as_index = False)
                r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
                n = n - 1
            d3 = pd.DataFrame(d2.X.min(), columns = ['min'])
            d3['min']=d2.min().X    
            d3['max'] = d2.max().X
            d3['sum'] = d2.sum().Y
            d3['total'] = d2.count().Y
            d3['bad_rate'] = d2.mean().Y
            d3['group_rate']=d3['total']/(bad+good)
            d3['woe']=np.log((d3['bad_rate']/(1-d3['bad_rate']))/(bad/good))
            d3['iv']=(d3['sum']/bad-((d3['total']-d3['sum'])/good))*d3['woe']
            iv=d3['iv'].sum()
            if iv!=0.0:
                woe=list(d3['woe'].round(5))
                cut=list(d3['min'].round(5))
                cut1=list(d3['max'].round(5))
                cut.append(cut1[-1]+1)
                x_woe=pd.cut(DF[i],cut,right=False,labels=woe)
                DF[i]=x_woe
            else:
                dn1 = pd.DataFrame({"X": DF[i], "Y": Y, "Bucket": pd.cut(DF[i], 20)})
                dn2 = dn1.groupby('Bucket', as_index = False)
                dn3 = pd.DataFrame(dn2.X.min(), columns = ['min'])
                dn3['min']=dn2.min().X    
                dn3['max'] = dn2.max().X
                dn3['sum'] = dn2.sum().Y
                dn3['total'] = dn2.count().Y
                dn3=dn3.dropna()
                dn3= dn3.reset_index(drop=True)
                while (1):
                    if  (len(dn3)>4):
                        dn3_min_index = dn3[dn3.total == min(dn3.total)].index.values[0]
                        if (dn3_min_index!=0):    #最小值非第一行的情况
                            dn3.iloc[dn3_min_index-1, 1] =dn3.iloc[dn3_min_index, 1] 
                            dn3.iloc[dn3_min_index-1, 2] = dn3.iloc[dn3_min_index, 2] +dn3.iloc[dn3_min_index-1, 2]
                            dn3.iloc[dn3_min_index-1, 3] = dn3.iloc[dn3_min_index, 3] +dn3.iloc[dn3_min_index-1, 3]
                            dn3=dn3.drop([dn3_min_index])
                            dn3= dn3.reset_index(drop=True)
                        else:    #最小值是第一行的情况
                            dn3.iloc[dn3_min_index+1, 0] =dn3.iloc[dn3_min_index, 0] 
                            dn3.iloc[dn3_min_index+1, 2] = dn3.iloc[dn3_min_index, 2] +dn3.iloc[dn3_min_index+1, 2]
                            dn3.iloc[dn3_min_index+1, 3] = dn3.iloc[dn3_min_index, 3] +dn3.iloc[dn3_min_index+1, 3]
                            dn3=dn3.drop([dn3_min_index])
                            dn3= dn3.reset_index(drop=True)
                    else:
                        break
                dn3['bad_rate'] =dn3['sum']/dn3['total']
                dn3['group_rate']=dn3['total']/(bad+good)
                dn3['woe']=np.log((dn3['bad_rate']/(1-dn3['bad_rate']))/(bad/good))
                dn3['iv']=(dn3['sum']/bad-((dn3['total']-dn3['sum'])/good))*dn3['woe']
                iv=dn3['iv'].sum()
                woe=list(dn3['woe'].round(5)) 
                cut=list(dn3['min'].round(5))
                cut1=list(dn3['max'].round(5))
                cut.append(cut1[-1]+1)
                x_woe=pd.cut(DF[i],cut,right=False,labels=woe)
                DF[i]=x_woe
        else : 
            bad=Y.sum()      #坏客户数
            good=Y.count()-bad  #好客户数
            d1 = pd.DataFrame({"X": DF[i], "Y": Y})
            d2 = d1.groupby('X', as_index =True)
            d3 = pd.DataFrame()
            d3['sum'] = d2.sum().Y
            d3['total'] = d2.count().Y
            for c in range(d3.shape[0])[::-1]:
                if ((d3.iloc[c,1]-d3.iloc[c,0])==0) or (d3.iloc[c,0]==0):
                    d3.iloc[c-1,0]=d3.iloc[c-1,0]+d3.iloc[c,0]
                    d3.iloc[c-1,1]=d3.iloc[c-1,1]+d3.iloc[c,1]
                    d3.drop(d3.index[c],inplace=True)
                else:
                    continue
            d3['bad_rate'] =d3['sum']/d3['total']
            d3['group_rate']=d3['total']/(bad+good)
            d3['woe']=np.log((d3['bad_rate']/(1-d3['bad_rate']))/(bad/good))
            d3['iv']=(d3['sum']/bad-((d3['total']-d3['sum'])/good))*d3['woe']
            iv=d3['iv'].sum()
            woe=list(d3['woe'].round(5))
            cut=list(d3.index)
            x_woe=DF[i].replace(cut,woe)
            DF[i]=x_woe
    return DF

保存到Excel本地

In [10]:
woe=woe_all(df,df.overdue_7,df.iloc[:,1:])   
woe.to_csv('文件路径.csv')

NameError: name 'df' is not defined

In [11]:
#目前单调分箱的连续型变量和类别变量基本与手动划分一致，高频型的连续变量还有待改进
#会报错类型Categorical categories must be unique
#可能是变量中存在离群值或者是无意义的变量